<a href="https://colab.research.google.com/github/anukvma/group18_email_subject_generation/blob/main/WIP_PK_QnA_ALBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_name = "twmkn9/albert-base-v2-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Load Dataset from CSV

In [ ]:
!git clone https://github.com/anukvma/group18_email_subject_generation


In [23]:
import pandas as pd
df = pd.read_csv('group18_email_subject_generation/sampled_qa_data.csv')
df.drop(columns=['Unnamed: 0', 'unit'], inplace=True)
df.head()

,Q,A
0,"What is Q-Learning, and how does it work?",Q-Learning is a model-free Reinforcement Learn...
1,Are there ethical considerations regarding AI-...,"Yes, concerns about data privacy, transparency..."
2,Explain the workflow of building a machine lea...,The process typically involves importing datas...
3,Can logistic regression be used for classes mo...,No logistic regression cannot be used for clas...
4,When does regularization come into play in Mac...,At times when the model begins to underfit or ...


In [24]:
import datasets
from datasets import Dataset, DatasetDict
dataset = Dataset.from_pandas(df)
def tokenize_function(examples):
    return tokenizer(
        examples["Q"],
        examples["A"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_datasets = dataset.map(tokenize_function)


Map:   0%|          | 0/667 [00:00<?, ? examples/s]

In [25]:
tokenized_datasets

Dataset({
    features: ['Q', 'A', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 667
})

In [27]:
def add_token_positions(examples):
    start_positions = []
    end_positions = []
    for i in range(len(examples["A"])):
        start_positions.append(examples["A"]["answer_start"][i])
        end_positions.append(examples["A"]["answer_start"][i] + len(examples["answers"]["text"][i]))
    examples["start_positions"] = start_positions
    examples["end_positions"] = end_positions
    return examples

tokenized_datasets = tokenized_datasets.map(add_token_positions, batched=True)

Map:   0%|          | 0/667 [00:00<?, ? examples/s]

TypeError: list indices must be integers or slices, not str